<a href="https://colab.research.google.com/github/DanyalKanzanAhmad/Project-02-LangChain-RAG-Project/blob/main/Project_02_LangChain_RAG_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [102]:
!pip install langchain_community pypdf -q langchain_google_genai  langchain_chroma -q

In [103]:
!pip install  --upgrade -q google-auth google-auth-oauthlib google-auth-httplib2  google-cloud


In [104]:
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

In [105]:
from langchain_community.document_loaders import PyPDFLoader

In [106]:
loader = PyPDFLoader("/content/Digital Products Ideas.pdf")
data = loader.load()
len(data)

5

In [107]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
docs = text_splitter.split_documents(data)
len(docs)

5

In [108]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/gen-lang-client-0213865457-8772de2adf53.json"


In [109]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [110]:
embedding_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001" , google_api_key=GOOGLE_API_KEY)


In [112]:
from langchain_chroma import Chroma
vectorstore = Chroma.from_documents(documents=docs,embedding = embedding_model)
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 10})

In [114]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model = "gemini-2.0-flash-exp" , temperature = 0.2 , max_tokens=1000)


In [115]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate




system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [116]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)


In [117]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)


In [119]:
response = rag_chain.invoke({"input" : "what is Digital Products"})
print(response["answer"])

Digital products include a variety of items such as video courses, audio tracks, graphic designs, templates, and documents. These products are often delivered electronically and can be used for educational, entertainment, or business purposes. Examples include ebooks, music, website themes, and social media branding kits.

